In [4]:
%run clean_data_en_lstm
%run clean_features

from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
#import duolingo_replica_alex as dr
#import duolingo_replica as d
import dask.dataframe as dd




time_steps_range = range(40, 201, 20)  # Time steps from 20 to 200 in steps of 20
mae_results = {}

for time_steps in time_steps_range:
  
    print(f"Training model with time_steps: {time_steps}")
        
    # time_steps = 20

    file_path = "https://www.dropbox.com/scl/fi/pnxa2jv4xf23bfwry1q9x/learning_traces.13m.csv?rlkey=2dt9848lutbgyys5sujq8dgw2&dl=1"
    result_df_en_top20 = process_data(file_path, time_steps)

    # Clean features 
    clean_data, feature_vars = read_data( result_df_en_top20, omit_lexemes = False )

    # Create time features 
    feature_vars_time = feature_vars + ['time_afternoon', 'time_evening', 'time_morning', 'time_night']

    import numpy as np
    from sklearn.model_selection import train_test_split

    # Assuming clean_data is your DataFrame, Y is the target column, and feature_vars are the features
    # Extract unique users
    unique_users = clean_data['user_id'].unique()  # Replace 'user_id' with your user identifier column

    # Split users into train and test groups
    train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)  # Adjust test_size as needed

    # Create training and testing datasets
    train_data = clean_data[clean_data['user_id'].isin(train_users)]
    test_data = clean_data[clean_data['user_id'].isin(test_users)]

    # Pre-sort the data by 'user_id' and 'datetime'
    train_data.sort_values(by=['user_id', 'datetime'], inplace= True)
    test_data.sort_values(by=['user_id', 'datetime'], inplace= True)

    # Create Batches so we cann run the model
    import numpy as np

    def create_non_overlapping_sequences(data, feature_vars, time_steps=time_steps, batch_size=1000):
        # Convert feature columns to numpy for faster processing
        feature_data = data[feature_vars].to_numpy()
        output_data = data['p'].to_numpy()  # Assuming 'p' is your target variable

        # Get the starting index for each new user
        user_change_indices = np.where(data['user_id'].to_numpy()[:-1] != data['user_id'].to_numpy()[1:])[0] + 1
        user_start_indices = np.insert(user_change_indices, 0, 0)

        for batch_start in range(0, len(user_start_indices), batch_size):
            sequences = []
            outputs = []

            # Iterate through each user in the batch
            for i in range(batch_start, min(batch_start + batch_size, len(user_start_indices))):
                start_idx = user_start_indices[i]
                end_idx = start_idx + time_steps if i + 1 < len(user_start_indices) else len(feature_data)

                # Check if the user data is exactly equal to time_steps
                if end_idx - start_idx == time_steps:
                    sequences.append(feature_data[start_idx:end_idx])
                    outputs.append(output_data[end_idx - 1])  # Target for the last period in the sequence

            yield np.array(sequences), np.array(outputs)

    # Complex model 
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras.regularizers import L1L2

    # # Enable mixed precision training
    # from tensorflow.keras.mixed_precision import experimental as mixed_precision
    # policy = mixed_precision.Policy('mixed_float16')
    # mixed_precision.set_policy(policy)

    # # Create and compile the model within a strategy scope if using multiple GPUs
    # strategy = tf.distribute.MirroredStrategy()

    # with strategy.scope():
    model = Sequential()
    model.add(LSTM(100, input_shape=(time_steps, len(feature_vars_time)), return_sequences=True, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu', return_sequences=False))
    model.add(Dense(1, kernel_regularizer=L1L2(l1=0.01, l2=0.01)))
    # Use experimental options for potential speed-up
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    from tensorflow.keras.metrics import MeanAbsoluteError
    import numpy as np

    # Training model 
    # Initialize the MAE metric
    mae_metric = MeanAbsoluteError()
    num_epochs = 20
    # Train the model using the generator
    for epoch in range(num_epochs):  # You can define the number of epochs
        print("Epoch {}/{}".format(epoch + 1, num_epochs))
        for X_batch, Y_batch in create_non_overlapping_sequences(train_data, feature_vars_time):
            # Train on batch and compute loss
            loss = model.train_on_batch(X_batch, Y_batch)

            # Compute MAE
            predictions = model.predict_on_batch(X_batch)
            mae_metric.update_state(Y_batch, predictions)
            mae = mae_metric.result().numpy()  # Get the MAE value

            # Reset MAE metric at the end of each batch
            mae_metric.reset_states()

            # Optionally print the loss and MAE for each batch
            print("Batch Loss: {:.4f}, Batch MAE: {:.4f}".format(loss, mae))

    # Initialize metric for testing
    test_mae_metric = MeanAbsoluteError()

    # Run model on test data
    for X_test_batch, Y_test_batch in create_non_overlapping_sequences(test_data, feature_vars_time):
        # Make predictions
        test_predictions = model.predict_on_batch(X_test_batch)

        # Update MAE metric
        test_mae_metric.update_state(Y_test_batch, test_predictions)
        print(test_mae_metric.result())

    # Calculate final MAE on test data
    final_test_mae = test_mae_metric.result().numpy()
    print("Final Test MAE: {:.6f}".format(final_test_mae))

    # Save the result
    mae_results[time_steps] = final_test_mae

# Save MAE results to a text file
with open('mae_results.txt', 'w') as file:
    for time_steps, mae in mae_results.items():
        file.write(f"{time_steps}: {mae}\n")

print("MAE results saved to mae_results.txt")


Training model with time_steps: 40


c:\Users\Alexander\anaconda3\envs\tensorflow\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Epoch 1/10
Batch Loss: 1.1178, Batch MAE: 0.8431
Batch Loss: 0.8620, Batch MAE: 0.7288
Batch Loss: 0.6354, Batch MAE: 0.5812
Batch Loss: 0.4412, Batch MAE: 0.4015
Batch Loss: 0.2713, Batch MAE: 0.2525
Batch Loss: 0.2244, Batch MAE: 0.3060
Batch Loss: 0.3261, Batch MAE: 0.2680
Batch Loss: 0.2814, Batch MAE: 0.2167
Batch Loss: 0.2127, Batch MAE: 0.2304
Batch Loss: 0.1927, Batch MAE: 0.2542
Batch Loss: 0.1936, Batch MAE: 0.2942
Batch Loss: 0.2271, Batch MAE: 0.3465
Batch Loss: 0.2330, Batch MAE: 0.3424
Batch Loss: 0.2185, Batch MAE: 0.3077
Batch Loss: 0.1993, Batch MAE: 0.2593
Batch Loss: 0.1783, Batch MAE: 0.2166
Batch Loss: 0.1816, Batch MAE: 0.2064
Epoch 2/10
Batch Loss: 0.1939, Batch MAE: 0.1855
Batch Loss: 0.1757, Batch MAE: 0.1492
Batch Loss: 0.2018, Batch MAE: 0.1690
Batch Loss: 0.1866, Batch MAE: 0.1470
Batch Loss: 0.1797, Batch MAE: 0.1550
Batch Loss: 0.1711, Batch MAE: 0.1656
Batch Loss: 0.1686, Batch MAE: 0.1900
Batch Loss: 0.1597, Batch MAE: 0.2028
Batch Loss: 0.1671, Batch MA

c:\Users\Alexander\anaconda3\envs\tensorflow\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Epoch 1/10
Batch Loss: 1.2090, Batch MAE: 0.8256
Batch Loss: 0.8994, Batch MAE: 0.6947
Batch Loss: 0.6591, Batch MAE: 0.5515
Batch Loss: 0.4346, Batch MAE: 0.3738
Batch Loss: 0.2765, Batch MAE: 0.2450
Batch Loss: 0.2519, Batch MAE: 0.3200
Batch Loss: 0.3043, Batch MAE: 0.2658
Batch Loss: 0.3214, Batch MAE: 0.2422
Batch Loss: 0.2504, Batch MAE: 0.2348
Batch Loss: 0.2228, Batch MAE: 0.3266
Batch Loss: 0.2418, Batch MAE: 0.3447
Batch Loss: 0.2415, Batch MAE: 0.3265
Batch Loss: 0.2208, Batch MAE: 0.2743
Batch Loss: 0.2128, Batch MAE: 0.2607
Epoch 2/10
Batch Loss: 0.2278, Batch MAE: 0.2500
Batch Loss: 0.1984, Batch MAE: 0.1997
Batch Loss: 0.1928, Batch MAE: 0.1795
Batch Loss: 0.2079, Batch MAE: 0.1801
Batch Loss: 0.1971, Batch MAE: 0.1699
Batch Loss: 0.1968, Batch MAE: 0.1748
Batch Loss: 0.1713, Batch MAE: 0.1678
Batch Loss: 0.1922, Batch MAE: 0.1969
Batch Loss: 0.1914, Batch MAE: 0.1993
Batch Loss: 0.1784, Batch MAE: 0.2188
Batch Loss: 0.1718, Batch MAE: 0.2092
Batch Loss: 0.1754, Batch MA

...

c:\Users\Alexander\anaconda3\envs\tensorflow\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Epoch 1/10
Batch Loss: 0.9828, Batch MAE: 0.7630
Batch Loss: 0.7550, Batch MAE: 0.6340
Batch Loss: 0.5199, Batch MAE: 0.4402
Batch Loss: 0.3328, Batch MAE: 0.2710
Batch Loss: 0.2354, Batch MAE: 0.2943
Batch Loss: 0.4942, Batch MAE: 0.2344
Batch Loss: 0.2104, Batch MAE: 0.2775
Batch Loss: 0.1953, Batch MAE: 0.2574
Batch Loss: 0.2088, Batch MAE: 0.2778
Epoch 2/10
Batch Loss: 0.2217, Batch MAE: 0.2803
Batch Loss: 0.1990, Batch MAE: 0.2682
Batch Loss: 0.1986, Batch MAE: 0.2590
Batch Loss: 0.1993, Batch MAE: 0.2548
Batch Loss: 0.1983, Batch MAE: 0.2479
Batch Loss: 0.1944, Batch MAE: 0.2349
Batch Loss: 0.1936, Batch MAE: 0.2392
Batch Loss: 0.1761, Batch MAE: 0.1904
Batch Loss: 0.1730, Batch MAE: 0.1693
Epoch 3/10
Batch Loss: 0.1969, Batch MAE: 0.1632
Batch Loss: 0.1801, Batch MAE: 0.1313
Batch Loss: 0.1870, Batch MAE: 0.1362
Batch Loss: 0.1839, Batch MAE: 0.1255
Batch Loss: 0.1890, Batch MAE: 0.1401
Batch Loss: 0.1810, Batch MAE: 0.1460
Batch Loss: 0.1677, Batch MAE: 0.1762
Batch Loss: 0.163

c:\Users\Alexander\anaconda3\envs\tensorflow\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Epoch 1/10
Batch Loss: 1.0297, Batch MAE: 0.7870
Batch Loss: 0.7725, Batch MAE: 0.6767
Batch Loss: 0.5680, Batch MAE: 0.5276
Batch Loss: 0.3983, Batch MAE: 0.3686
Batch Loss: 0.2537, Batch MAE: 0.2394
Batch Loss: 0.2403, Batch MAE: 0.2893
Batch Loss: 0.3190, Batch MAE: 0.2947
Batch Loss: 0.3697, Batch MAE: 0.2652
Epoch 2/10
Batch Loss: 0.4013, Batch MAE: 0.2674
Batch Loss: 0.2282, Batch MAE: 0.2491
Batch Loss: 0.2104, Batch MAE: 0.3008
Batch Loss: 0.2152, Batch MAE: 0.3435
Batch Loss: 0.2256, Batch MAE: 0.3640
Batch Loss: 0.2741, Batch MAE: 0.4108
Batch Loss: 0.2705, Batch MAE: 0.3963
Batch Loss: 0.2576, Batch MAE: 0.3698
Epoch 3/10
Batch Loss: 0.2384, Batch MAE: 0.3348
Batch Loss: 0.2235, Batch MAE: 0.3083
Batch Loss: 0.2142, Batch MAE: 0.2795
Batch Loss: 0.1988, Batch MAE: 0.2466
Batch Loss: 0.1780, Batch MAE: 0.1985
Batch Loss: 0.1864, Batch MAE: 0.2069
Batch Loss: 0.1787, Batch MAE: 0.1716
Batch Loss: 0.1785, Batch MAE: 0.1648
Epoch 4/10
Batch Loss: 0.2141, Batch MAE: 0.1784
Batch 

MemoryError: Unable to allocate 24.2 GiB for an array with shape (2508, 1295680) and data type float64

In [ ]:
print("Alex")

Alex
